In [1]:
# Use this to add the tag_tracking folder to python path for imports
import os, sys
tag_tracking_path = os.path.abspath(os.path.join('', './../..'))
sys.path.insert(0, tag_tracking_path)
# If you get an error here about no "pygrid" module, you need to compile some C code:
#    In the tag_tracking/tagsim directory, run
#        python setup.py build_ext --inplace
#    or
#        python setup_xcode.py build_ext --inplace   # (if you are using xcode on a Mac)


# Import the tagsim functions
from tagsim import SimObject, SimPSD
from tagsim.sim_fullmotion import proc_im

import numpy as np
import matplotlib.pyplot as plt
import pydicom
from copy import deepcopy

In [2]:
ke = .08  # DENSE encoding strength
use_gpu = False  # Whether to use GPU (not really needed for this small example)

Nt = 25  # Number of timeframes

N_im = 32  # Image resolution (FOV = 80 mm is hardcoded for this example)

N_phasecycle = 3 # Number of DENSE phasecycles (could try 2 here, but echo cancellation will probably be incomplete)

# How much noise to add.  TODO: correspond this number to SNR, right now it is arbitrary 
noise_scale = 0.5 

# Whether to apply Hamming filter or not.  (This blurs image, it may be more reaslistic to have it on)
do_hamming = False

# Compute what time each of the images correspond to in milliseconds
acq_loc = np.arange(0, Nt) * 1000 / Nt + 1  


# Prep the structure (time resolved simulation points ready for simulation)
sim_object = SimObject.SimObject()
sim_object.gen_solo_cardiac()


# We are going to loop though all encoding directions now
dir_all_im = []
dir_all_im_pc = []

for enc_dir in range(3):

    ke_dir = np.zeros(3)  # DENSE encoding direction
    ke_dir[enc_dir] = 1.0
    print('Running sim for ke_dir = {}'.format(str(ke_dir)))

    # Phase cycling RF angles
    all_theta = np.linspace(0, 2*np.pi, N_phasecycle, endpoint=False)
    all_acq = []

    print('Running Bloch simulator for theta =', end = ' ', flush = True)
    for theta in all_theta:
        print('{:.2f}'.format(theta), end = ' ', flush = True)
        # Preps the simulator
        simulator = SimPSD.SimInstant(sim_object, use_gpu=use_gpu)
        # Prep the DENSE PSD
        simulator.sample_DENSE_PSD(rf_dir = [np.cos(theta), np.sin(theta), 0], ke=ke, ke_dir=ke_dir, kd = 0.0, acq_loc=acq_loc)
        # Run simulator
        all_acq.append(simulator.run())
    print('Done!')

    print('Solving for images . . .', end = ' ', flush = True)
    # This code builds the images from the simulated complex points and position.
    all_im = np.zeros((N_phasecycle, Nt, N_im, N_im), np.complex)
    for i_pc in range(N_phasecycle):
        for i_t in range(Nt):
            im0 = sim_object.grid_im_from_M(all_acq[i_pc][i_t][0], all_acq[i_pc][i_t][1], N_im=N_im, w=N_im // 2, use_gpu = use_gpu) #, dens = dd)
            im0 = proc_im(im0, N_im, noise_scale=noise_scale, kaiser_beta=0, do_hamming = do_hamming)
            all_im[i_pc, i_t] = im0
    dir_all_im.append(all_im)

    # Combines the phase cycled image into a single (NtxNxN) final image array (all_im_pc)
    all_im_pc = all_im.copy()
    all_im_pc *= np.conj(np.exp(1j * all_theta))[:, None, None, None]
    all_im_pc = all_im_pc.sum(0)
    dir_all_im_pc.append(all_im_pc)

    print('Done!')

dir_all_im = np.array(dir_all_im)
dir_all_im_pc = np.array(dir_all_im_pc)

Running sim for ke_dir = [1. 0. 0.]
Running Bloch simulator for theta = 0.00 2.09 4.19 Done!
Solving for images . . . Done!
Running sim for ke_dir = [0. 1. 0.]
Running Bloch simulator for theta = 0.00 2.09 4.19 Done!
Solving for images . . . Done!
Running sim for ke_dir = [0. 0. 1.]
Running Bloch simulator for theta = 0.00 2.09 4.19 Done!
Solving for images . . . Done!


In [3]:
mag = np.mean(np.abs(dir_all_im_pc), axis=0)
px = np.angle(dir_all_im_pc[0])
py = np.angle(dir_all_im_pc[1])
pz = np.angle(dir_all_im_pc[2])
p_all = np.angle(dir_all_im_pc)

print(p_all.shape, p_all.dtype, p_all.min(), p_all.max())

(3, 25, 32, 32) float64 -3.141585498858447 3.1415549157605005


In [9]:
# Load template dicom file
template_ds = pydicom.dcmread('../cardiac_data/template.dcm')

# Don't use lossless compression to make things a little easier to work with
template_ds.decompress()
del template_ds.DerivationDescription

save_dir = '../cardiac_data/dense_dicom/'
os.makedirs(save_dir, exist_ok=True)
all_sub_dir = ['px', 'py', 'pz', 'mag']
for sub_dir in all_sub_dir:
    os.makedirs(os.path.join(save_dir, sub_dir), exist_ok=True)


all_series_no = [2, 3, 4, 1]    
Nt = mag.shape[0]



for enc_dir in range(4):

    series_no = all_series_no[enc_dir]
    series_uid = pydicom.uid.generate_uid()

    if enc_dir == 0:
        tag0 = 'x-encPha'
        tag1 = 'x-enc pha'
    elif enc_dir == 1:
        tag0 = 'y-encPha'
        tag1 = 'y-enc pha'
    elif enc_dir == 2:
        tag0 = 'z-encPha'
        tag1 = 'z-enc pha'
    elif enc_dir == 3:
        tag0 = 'AveMag'
        tag1 = 'overall mag'
    
    for it in range(Nt):
        
        if enc_dir < 3:
            comment = 'DENSE {} - Scale:1.00000 EncFreq:{:.2f} Rep:0/1 Slc:0/1 Par:{:d}/{:d} Phs:{:d}/{:d} RCswap:1 RCSflip:0/0/0'.format(
                tag1, ke, 0, 1, it, Nt
                )
        elif enc_dir == 3:
            comment = 'DENSE {} - Rep:0/1 Slc:0/1 Par:{:d}/{:d} Phs:{:d}/{:d} RCswap:1 RCSflip:0/0/0'.format(
                tag1, 0, 1, it, Nt
            )

        series_desc = 'Computational Phantom Series {:02d} {}'.format(series_no, tag0)

        ds = deepcopy(template_ds)

        ds.SeriesDescription = series_desc
        ds.SeriesInstanceUID = series_uid
        ds.CardiacNumberOfImages = Nt
        ds.SeriesNumber = series_no
        ds.InstanceNumber = it
        ds.ImageComments = comment

        if enc_dir < 3:
            im = p_all[enc_dir, it]
            im = (im/np.pi * 2048 + 2048).astype(np.uint16)
        elif enc_dir == 3:
            im = mag[it]
            im = (4096*(im/mag.max())).astype(np.uint16)
        

        ds.PixelData = im.T.tostring()

        fname = 'im_{:03d}.dcm'.format(it)
        ds.save_as( os.path.join(save_dir, all_sub_dir[enc_dir], fname) )
        
        

